# Neural networks, Checkpoint 

Faculty: FIIT <br>
Academic year: 2019/2020 <br>
Authors: Andrej Gáfrik, Martin Grega <br>


We had to change out source data to https://figshare.com/articles/Wikipedia_Detox_Data/4054689, because of bad formatting in data, which we couldn't solve.

In [15]:
# !pip install pandas
# !pip install nltk
# !pip install tokenizer
#!pip uninstall keras
#!pip install keras --upgrade
!pip install sklearn

     |████████████████████████████████| 7.0MB 9.1MB/s eta 0:00:01
     |████████████████████████████████| 296kB 45.7MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=769d91b7b17a4c4a3775deb75baf2b09b1ef1b7c85f36afe2ff6de92151bf32d
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
comments = pd.read_csv('../../data/attack_annotated_comments.tsv', sep='\t')
labels = pd.read_csv('../../data/attack_annotations.tsv', sep='\t')

In [ ]:
comments = comments.drop(columns=['year','logged_in','ns','sample','split'])

In [ ]:
comments

Problems in data:
 - tabular, new line and qoutes were replaced with tab_token, new_line token and `
 - punctuation
 - more spaces between words
 - empty strings
 - abbreviations like i will = i'll , didn't = did not, ...

In [ ]:
labels = labels.drop(columns=['worker_id','quoting_attack','recipient_attack','third_party_attack','other_attack'])

In [ ]:
labels



In [ ]:
res = labels.groupby('rev_id')['attack'].mean().reset_index()

In [ ]:
res

In [ ]:
data = comments.join(res.set_index('rev_id'), on='rev_id')
data['attack'] = data['attack'].round(0)

In [ ]:
data

In [ ]:
data.info()
data.isnull().values.any()

In [ ]:
# In out data are newlines, tabulators and quotions marks replaced, so we need to delete this replacement
data['comment'] = data['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN",""))
data['comment'] = data['comment'].apply(lambda x: x.replace("TAB_TOKEN",""))
data['comment'] = data['comment'].apply(lambda x: x.replace("``",'"'))

In [ ]:
# Source: https://www.kaggle.com/prashantkikani/pooled-gru-with-preprocessing
repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "'s" : " is",
    "'s" : " is",
    "`d": " would",
    "that's" : "that is",
    "weren't" : "were not",
    "e.g" : "eg",
}

In [ ]:
#splitting comments by spaces
comments = data['comment'].apply(lambda x: x.split())

In [ ]:
import re
abbr = [i for i in repl.keys()]
result = []
for index in range(len(comments)):
    new_comment = ""
    for word in comments[index]:
        word = word.lower()
        word = re.sub(r"n't", " not", word)
        word = re.sub(r"\ 's", " not", word)
        word = re.sub(r"\ 've", " not", word)
        word = re.sub(r"\ 'd", " not", word)
        word = re.sub(r"\ ll", " not", word)
        if re.search("^http.+|^www.+",word): # deleting links
            continue
        elif word in abbr:
            new_comment += repl[word]
            new_comment += " "
            word = repl[word]
        elif re.search("[^a-zA-Z ]+",word): # only alphabet
            new_comment += re.sub("[^a-zA-Z ]+"," ",word)
            new_comment += " "
        else:
            new_comment += word  
            new_comment += " "
            
    new_comment = " ".join(new_comment.split())        
   # print("Old = ", comments[index])
   # print("New = ", new_comment)
    result.append(new_comment)
data["comment"] = result

In [ ]:
pd.options.display.max_rows = 999
data

In [ ]:
# replacing empty strings with nan
data['comment'].replace('', np.nan, inplace=True)

In [ ]:
# checking nan values on data
data.info()
data.isnull().values.any()

In [ ]:
# drop nan rows
data.dropna(subset=['comment'], inplace=True)
data = data.reset_index(drop=True)

# Stemming

In [ ]:
from nltk.stem import SnowballStemmer

def stemmer(comment):
    stemmer = SnowballStemmer('english')
    words = comment.split()
    stemmed = [stemmer.stem(word) for word in words]
    stemmed
    return " ".join(stemmed)

t = data['comment'].map(lambda x: stemmer(x))

In [ ]:
data['stemmed_comments'] = t

In [66]:
data['number_of_words'] = data.stemmed_comments.apply(lambda x: len(x.split()))

In [89]:
data = data.loc[data['number_of_words'] <= 300]

In [90]:
data.loc[data['number_of_words'].idxmax()]

rev_id                                                        6360046
comment             wikipedia is an encyclopedia not a dictionary ...
attack                                                              0
stemmed_comments    wikipedia is an encyclopedia not a dictionari ...
number_of_words                                                   300
Name: 479, dtype: object

In [102]:
data['attack'].value_counts()

0.0    98887
1.0    13206
Name: attack, dtype: int64

In [101]:
# saving cleaned data
data.to_csv('../../data/preprocessed_data_1.csv', encoding='utf-8')

# Tokenizing

In [2]:
data= pd.read_csv('../../data/preprocessed_data_1.csv', index_col=0)

In [3]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

In [4]:
tokenizer = Tokenizer()

# Training of our tokenizer
tokenizer.fit_on_texts(data['stemmed_comments'])
#tokenizer.fit_on_texts(data['comment'])

# Number of unique words in our data
vocab_len = len(tokenizer.word_index) + 1

#converting comment into numeric form, each unique word has a number, so comment will be rewrite into numbers
embedded = tokenizer.texts_to_sequences(data['stemmed_comments'])

In [5]:
longest_sentence_len = len(max(embedded, key= len))

#Adding zeroes to end of eat embeded sentece to len of the longest sentence
padded_comments = pad_sequences(embedded, longest_sentence_len, padding='post')

# Glove embeddings

In [6]:
glove = open('../../data/glove.6B.100d.txt', encoding="utf8")
from numpy import array
from numpy import asarray

In [7]:
# Train and test data
size_train = round(len(padded_comments)* 0.8)

train_padded = padded_comments[:size_train]
train_attack =asarray(data['attack'][:size_train], dtype='int32')

test_padded = padded_comments[size_train + 1:]
test_attack = asarray(data['attack'][size_train + 1:], dtype='int32')

In [8]:
# Preparing glove dictonary
glove_dict = dict()

#according to glove file, first is word and next are vectors
for line in glove:
    splitted = line.split()
    word = splitted[0]
    vectors = asarray(splitted[1:], dtype='float32')
    glove_dict [word] = vectors 


glove.close()

In [9]:
from numpy import zeros
zle = 0
# create dict with only words in out data
word_matrix = zeros((vocab_len, 100))
# iterate throuh all words in our data and find vector for them
for word, index in tokenizer.word_index.items():
    vectors = glove_dict.get(word)
    if vectors is not None:
        word_matrix[index] = vectors # pretrained word embedings with words from our comments
        

# GRU 

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense
from tensorflow.keras.initializers import Constant
from sklearn.utils import class_weight

def GRU_model():
    model = Sequential()
    embedding_layer = Embedding(vocab_len,100,embeddings_initializer = Constant(word_matrix),input_length=longest_sentence_len,trainable=False)
    model.add(embedding_layer)
    model.add(GRU(units = 64, dropout=0.2, recurrent_dropout = 0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model


In [11]:
model = GRU_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          9365400   
_________________________________________________________________
gru (GRU)                    (None, 64)                31872     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 9,397,337
Trainable params: 31,937
Non-trainable params: 9,365,400
_________________________________________________________________


In [12]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_attack),
                                                 train_attack)

In [13]:
np.unique(train_attack, return_counts=True)

(array([0, 1], dtype=int32), array([78913, 10761]))

In [14]:
weights = {0: class_weights[0], 1: class_weights[1]}
weights

{0: 0.5681826821943152, 1: 4.166620202583403}

In [15]:
#Not working in notebook and docker, trained in google colab, screen of results in model
model.fit(train_padded,train_attack,batch_size=128, epochs=3, 
          validation_data=(test_padded,test_attack),verbose=1, class_weight=weights)

Train on 89674 samples, validate on 22418 samples
Epoch 1/3
89674/89674 [==============================] - 307s 3ms/sample - loss: 0.3544 - accuracy: 0.6048 - val_loss: 0.2555 - val_accuracy: 0.1100
Epoch 2/3
89674/89674 [==============================] - 304s 3ms/sample - loss: 0.3930 - accuracy: 0.7314 - val_loss: 0.2550 - val_accuracy: 0.1100
Epoch 3/3
89674/89674 [==============================] - 303s 3ms/sample - loss: 0.4120 - accuracy: 0.7593 - val_loss: 0.2582 - val_accuracy: 0.1100


Results of training (because used wrong verbose parameter :( )
- Epoch 1: 1110S - loss: 0.4846 - acc: 0.8798 - val_loss: 0.3513 - val_acc: 0.8951
- Epoch 2: 1284S - loss: 0.3699 - acc: 0.8798 - val_loss: 0.3373 - val_acc: 0.8951
- Epoch 3: 1290S - loss: 0.3681 - acc: 0.8798 - val_loss: 0.3571 - val_acc: 0.8951

In [ ]:
model.save('GRU_model_1') 

# SimpleRNN

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,SimpleRNN
from tensorflow.keras.initializers import Constant

def Simple_RNN():
    model = Sequential()
    embedding_layer = Embedding(vocab_len,100,embeddings_initializer = Constant(word_matrix),input_length=longest_sentence_len,trainable=False)
    model.add(embedding_layer)
    model.add(SimpleRNN(128,activation = 'tanh'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
model2 = Simple_RNN()
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          9365400   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               29312     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 9,394,841
Trainable params: 29,441
Non-trainable params: 9,365,400
_________________________________________________________________


In [18]:
model2.fit(train_padded,train_attack,batch_size=128, epochs=3, 
           validation_data=(test_padded,test_attack),verbose=1, class_weight=weights)

Train on 89674 samples, validate on 22418 samples
Epoch 1/3
89674/89674 [==============================] - 139s 2ms/sample - loss: 0.2495 - accuracy: 0.4107 - val_loss: 0.2368 - val_accuracy: 0.8909
Epoch 2/3
89674/89674 [==============================] - 139s 2ms/sample - loss: 0.2468 - accuracy: 0.4029 - val_loss: 0.2415 - val_accuracy: 0.2359
Epoch 3/3
89674/89674 [==============================] - 138s 2ms/sample - loss: 0.2473 - accuracy: 0.4103 - val_loss: 0.2442 - val_accuracy: 0.2320


In [ ]:
model2.save('Simple_RNN_1')

# LSTM model

In [65]:
def LSTM():
    model = Sequential()
    embedding_layer = Embedding(vocab_len,100,embeddings_initializer = Constant(word_matrix),input_length=longest_sentence_len,trainable=False)
    model.add(embedding_layer)
#     model.add(LSTM())
#     model.add(GRU(64))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model3 = LSTM()
model3.summary()